In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [20]:
# Load association results:

association_results_df = pd.read_csv('../data/20220429_association_results_wardrobesize_2-20_wo_none.csv', \
    index_col=0,  dtype={'antecedants':'string', 'consequents':'string'})

association_results_df.sort_values('lift', ascending=False, inplace=True)
association_results_df

In [26]:
# Generate wardrobe table:
df_trans = pd.read_csv('../data/transactions_train.csv', dtype={'article_id':'string'})
df_trans_red = df_trans.drop(columns=['t_dat', 'price', 'sales_channel_id']).copy()
df_wardrobe = df_trans_red.groupby('customer_id')['article_id'].aggregate(lambda x: list(x)).reset_index()


In [38]:
# Clear memory
del [[df_trans,df_trans_red]]

In [39]:
df_wardrobe.head()


,customer_id,article_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,"[0625548001, 0176209023, 0627759010, 069713800..."
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,"[0583558001, 0639677008, 0640244003, 052126900..."
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,"[0663713001, 0541518023, 0663713001, 057802000..."
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,"[0742079001, 0732413001]"
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,"[0634249005, 0677049001, 0698286003, 070770400..."


In [43]:
# Add 'number of articles' column (needed to generate smaller dataset) and sort descending:

df_wardrobe['no_articles'] = df_wardrobe.article_id.apply(lambda x: len(x))
df_wardrobe.sort_values('no_articles', ascending=False, inplace=True)

In [144]:
# Select a sample for testing code
# df_sample = df_wardrobe.sample(n=1000000, random_state=42)
# df_sample.sort_values('no_articles', ascending=False)

df_sample = df_wardrobe

In [145]:
# Generate empty column for recommendation
df_sample['reco'] = df_sample.apply(lambda x: [], axis=1)

In [146]:
# Append consequents as list

for i in range(len(df_sample)):
    for j in range(len(association_results_df)):
        if association_results_df.antecedants.iloc[j] in df_sample.article_id.iloc[i]:
            if association_results_df.consequents.iloc[j] not in df_sample.article_id.iloc[i]:
                df_sample['reco'].iloc[i].append(association_results_df.consequents.iloc[j])
            

# sort consequents by lift-value

In [1]:
df_sample['no_reco'] = df_sample.reco.apply(lambda x: len(x))
df_sample.sort_values('no_reco', ascending=False).head()


NameError: name 'df_sample' is not defined

In [151]:
# Number of customer_ids with at least 1 recommendation:
df_sample[df_sample['reco_len']>0].shape[0]

324638

In [152]:
# Store recommendations in csv:
df_sample.to_csv('../data/20220429_all_cust_recommendations_wardrobesize_2-20.csv')